# Create zonal means and save to file

In [1]:
import numpy as np
import numpy.matlib
import numpy.ma as ma

import xarray as xr

import time
import cftime
import netCDF4 as nc
from datetime import timedelta

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt


import glob


## notes on packages to add to this kernel
import nc_time_axis
import cartopy.crs as ccrs
import cartopy



In [2]:
# load custom functions for analyzing flat10


from loading_function_flat10 import load_one_model_onevar, load_grid, select_time_slice, weighted_temporal_mean 


In [3]:
outputdir= '/glade/campaign/cgd/tss/people/aswann/flat10/'

modellist_orig= ['ACCESS-ESM1-5',  
            'CESM2',    
            'GFDL-ESM4',  
            'GISS_E2.1',  
            'NorESM2-LM',
            'MPI-ESM1-2-LR',
            'CNRM-ESM2-1',
            'HadCM3LC-Bris',
            'UKESM1.2',
            'MIROC-ES2L']
modellist=modellist_orig

runlist = ['flat10','flat10_zec','flat10_cdr']
# use a wildcard to capture different ways the folders and runs are named across models
runlist_wc = ['*lat10','*zec','*cdr']

varlist_load=['cVeg','cSoil','cLitter','nbp','gpp','rh'] #, 'gpp','fgco2', 'ra', 'rh']#, 'npp'] # not working beyond nbp for norESM
varlist_analyze=['cVeg','cSoil','cTot','cLitter','nbp','gpp','rh']
varlist=varlist_load
unitslist=['kgC m-2','kgC m-2','kgC m-2','kgC m-2 s-1','kgC m-2 s-1','kgC m-2 s-1']

# there seems to be a problem with ra for NorESM

modelcolors=['tab:blue','tab:orange','tab:green','tab:red','tab:gray','tab:purple','tab:cyan','gold','tab:brown']
### from ben: colors=["tab:cyan","tab:olive","tab:green","tab:red","tab:gray","tab:pink","limegreen","tab:brown", "slateblue","gold"]


# to match Sanderson paper:
modellist_ben=['ACCESS-ESM1-5',
                'CESM2',
                'GFDL-ESM4',
                'GISS_E2.1',
                'NorESM2-LM',
                'MPI-ESM1-2-LR',
                'CNRM-ESM2-1',
                'UKESM1.2',
                # 'UVic-ESCM-2-10',
                'HadCM3LC-Bris',
                'MIROC-ES2L']
# bencolors=["tab:cyan","tab:olive","tab:green","tab:red","tab:gray","tab:pink","limegreen","tab:brown", "slateblue","gold"]
bencolors=["tab:cyan","tab:olive","tab:green","tab:red","tab:gray","tab:pink","limegreen","tab:brown", "gold", "tab:purple"]

# -----select either original colors or ben's colors and order
modellist=modellist_ben
modelcolors=bencolors


In [4]:
#unitconversions
sperday=60*60*24
speryr=60*60*24*365

PgperKg = 1e-12
# to convert kg m-2 s-1 to kg/m2/yr multiply by speryr

In [5]:
# initialize a dictionary to hold all of the data
data_dict={}

# load grid info

In [6]:
data_dict = load_grid(data_dict,modellist)

ACCESS-ESM1-5 getting grid info
CESM2 getting grid info
GFDL-ESM4 getting grid info
GISS_E2.1 getting grid info
NorESM2-LM getting grid info
MPI-ESM1-2-LR getting grid info
CNRM-ESM2-1 getting grid info
UKESM1.2 getting grid info
HadCM3LC-Bris getting grid info
MIROC-ES2L getting grid info


# load and save to netcdf

In [7]:
# phasename=['InitialC','EmissionsPhase','NetZero','CumulativeZero']
# modellist=['MIROC-ES2L']

unitconvert=PgperKg

# loop over models
for m in range(len(modellist)):
    model=modellist[m]
    print('Loading ' +model)

    # these will be loaded right here
    ds0g=load_one_model_onevar(model,runlist_wc[0],'cVeg')

    ###---- grid info
    lat = ds0g.lat

    #--- get area and land fraction
    ds_area = data_dict[modellist[m] +'_' +'areacella']
    ds_landfrac = data_dict[modellist[m] +'_' +'landfrac']
    
    if model=='CESM2':
        area = ds_area['areacella'].squeeze().reindex_like(ds0g, method='nearest',tolerance=0.05)
    else:
        area = ds_area['areacella'].reindex_like(ds0g, method='nearest',tolerance=0.05)
    
    landfrac=ds_landfrac['sftlf'].reindex_like(ds0g, method='nearest',tolerance=0.05)
    
    if landfrac.max(dim=['lat','lon'])>1: #test if landfrac is on a 0-100 or 0-1 scale
        landfrac=landfrac/100

    landarea=(area*landfrac)
    area_per_lat_m2=(area*landfrac).sum(dim='lon')

    #----###-----####----- cVeg
    var='cVeg'
    ds0=load_one_model_onevar(model,runlist_wc[0],var) #load one model one experiment one var
    # ds0=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds0,var) # annual mean    
    # del ds # drop full grid from memory   
    dst= select_time_slice(dsw, 0, 9) # first ten years of flat10
    dsften_cVeg = dst.mean(dim='time').mean(dim='lon') # original units
    dsften_cVeg_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time') # converted to PgC
    dst=select_time_slice(dsw, 90, 99) # last ten years of flat10
    dslten0_cVeg = dst.mean(dim='time').mean(dim='lon') # original units
    dslten0_cVeg_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds1=load_one_model_onevar(model,runlist_wc[1],var) #load one model one experiment one var
    # ds1=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds1,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) #last ten years of flat10-zec
    dslten1_cVeg = dst.mean(dim='time').mean(dim='lon') # original units
    dslten1_cVeg_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds2=load_one_model_onevar(model,runlist_wc[2],var) #load one model one experiment one var
    # ds2=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds2,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) # last ten years of flat10-cdr
    dslten2_cVeg = dst.mean(dim='time').mean(dim='lon') # original units
    dslten2_cVeg_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC


    #----###-----####----- cSoil
    var='cSoil'
    ds0=load_one_model_onevar(model,runlist_wc[0],var) #load one model one experiment one var
    # ds0=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds0,var) # annual mean    
    # del ds # drop full grid from memory   
    dst= select_time_slice(dsw, 0, 9) # first ten years of flat10
    dsften_cSoil = dst.mean(dim='time').mean(dim='lon') # original units
    dsften_cSoil_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time') # converted to PgC
    dst=select_time_slice(dsw, 90, 99) # last ten years of flat10
    dslten0_cSoil = dst.mean(dim='time').mean(dim='lon') # original units
    dslten0_cSoil_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds1=load_one_model_onevar(model,runlist_wc[1],var) #load one model one experiment one var
    # ds1=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds1,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) #last ten years of flat10-zec
    dslten1_cSoil = dst.mean(dim='time').mean(dim='lon') # original units
    dslten1_cSoil_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds2=load_one_model_onevar(model,runlist_wc[2],var) #load one model one experiment one var
    # ds2=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds2,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) # last ten years of flat10-cdr
    dslten2_cSoil = dst.mean(dim='time').mean(dim='lon') # original units
    dslten2_cSoil_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC


    #----###-----####----- cLitter
    var='cLitter'
    ds0=load_one_model_onevar(model,runlist_wc[0],var) #load one model one experiment one var
    if ds0 is not None:
        litterflag=1
        # ds0=ds.mean(dim='lon') # zonal mean
        dsw=weighted_temporal_mean(ds0,var) # annual mean    
        # del ds # drop full grid from memory   
        dst= select_time_slice(dsw, 0, 9) # first ten years of flat10
        dsften_cLitter = dst.mean(dim='time').mean(dim='lon') # original units
        dsften_cLitter_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time') # converted to PgC
        dst=select_time_slice(dsw, 90, 99) # last ten years of flat10
        dslten0_cLitter = dst.mean(dim='time').mean(dim='lon') # original units
        dslten0_cLitter_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC
    
        ds1=load_one_model_onevar(model,runlist_wc[1],var) #load one model one experiment one var
        # ds1=ds.mean(dim='lon') # zonal mean
        dsw=weighted_temporal_mean(ds1,var)
        # del ds # drop full grid from memory
        dst = select_time_slice(dsw, 190, 199) #last ten years of flat10-zec
        dslten1_cLitter = dst.mean(dim='time').mean(dim='lon') # original units
        dslten1_cLitter_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC
    
        ds2=load_one_model_onevar(model,runlist_wc[2],var) #load one model one experiment one var
        # ds2=ds.mean(dim='lon') # zonal mean
        dsw=weighted_temporal_mean(ds2,var)
        # del ds # drop full grid from memory
        dst = select_time_slice(dsw, 190, 199) # last ten years of flat10-cdr
        dslten2_cLitter = dst .mean(dim='time').mean(dim='lon') # original units
        dslten2_cLitter_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC
    
    
        #----###-----####----- cTot
        # derived variable, sum of Veg, Soil, Litter
        # var='cTot'
        dsften_cTot = dsften_cVeg + dsften_cSoil + dsften_cLitter
        dsften_cTot_PgC = dsften_cVeg_PgC + dsften_cSoil_PgC + dsften_cLitter_PgC
        dslten0_cTot = dslten0_cVeg + dslten0_cSoil + dslten0_cLitter
        dslten0_cTot_PgC = dslten0_cVeg_PgC + dslten0_cSoil_PgC + dslten0_cLitter_PgC
        dslten1_cTot = dslten1_cVeg + dslten1_cSoil + dslten1_cLitter
        dslten1_cTot_PgC = dslten1_cVeg_PgC + dslten1_cSoil_PgC + dslten1_cLitter_PgC
        dslten2_cTot = dslten2_cVeg + dslten2_cSoil + dslten2_cLitter
        dslten2_cTot_PgC = dslten2_cVeg_PgC + dslten2_cSoil_PgC + dslten2_cLitter_PgC

    else: # litter variable doesn't exist
        litterflag=0
        #----###-----####----- cTot
        # derived variable, sum of Veg, Soil, but NOT litter
        # var='cTot'
        dsften_cTot = dsften_cVeg + dsften_cSoil
        dsften_cTot_PgC = dsften_cVeg_PgC + dsften_cSoil_PgC
        dslten0_cTot = dslten0_cVeg + dslten0_cSoil 
        dslten0_cTot_PgC = dslten0_cVeg_PgC + dslten0_cSoil_PgC 
        dslten1_cTot = dslten1_cVeg + dslten1_cSoil 
        dslten1_cTot_PgC = dslten1_cVeg_PgC + dslten1_cSoil_PgC 
        dslten2_cTot = dslten2_cVeg + dslten2_cSoil 
        dslten2_cTot_PgC = dslten2_cVeg_PgC + dslten2_cSoil_PgC 


    #####---- Fluxes------###
    unitconvert=speryr*PgperKg
    
    #----###-----####----- gpp
    var='gpp'
    ds0=load_one_model_onevar(model,runlist_wc[0],var) #load one model one experiment one var
    # ds0=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds0,var) # annual mean    
    # del ds # drop full grid from memory   
    dst= select_time_slice(dsw, 0, 9) # first ten years of flat10
    dsften_gpp = dst.mean(dim='time').mean(dim='lon') # original units
    dsften_gpp_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time') # converted to PgC
    dst=select_time_slice(dsw, 90, 99) # last ten years of flat10
    dslten0_gpp = dst.mean(dim='time').mean(dim='lon') # original units
    dslten0_gpp_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds1=load_one_model_onevar(model,runlist_wc[1],var) #load one model one experiment one var
    # ds1=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds1,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) #last ten years of flat10-zec
    dslten1_gpp = dst.mean(dim='time').mean(dim='lon') # original units
    dslten1_gpp_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds2=load_one_model_onevar(model,runlist_wc[2],var) #load one model one experiment one var
    # ds2=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds2,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) # last ten years of flat10-cdr
    dslten2_gpp = dst.mean(dim='time').mean(dim='lon') # original units
    dslten2_gpp_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    #----###-----####----- nbp
    var='nbp'
    ds0=load_one_model_onevar(model,runlist_wc[0],var) #load one model one experiment one var
    # ds0=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds0,var) # annual mean    
    # del ds # drop full grid from memory   
    dst= select_time_slice(dsw, 0, 9) # first ten years of flat10
    dsften_nbp = dst.mean(dim='time').mean(dim='lon') # original units
    dsften_nbp_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time') # converted to PgC
    dst=select_time_slice(dsw, 90, 99) # last ten years of flat10
    dslten0_nbp = dst.mean(dim='time').mean(dim='lon') # original units
    dslten0_nbp_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds1=load_one_model_onevar(model,runlist_wc[1],var) #load one model one experiment one var
    # ds1=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds1,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) #last ten years of flat10-zec
    dslten1_nbp = dst.mean(dim='time').mean(dim='lon') # original units
    dslten1_nbp_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds2=load_one_model_onevar(model,runlist_wc[2],var) #load one model one experiment one var
    # ds2=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds2,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) # last ten years of flat10-cdr
    dslten2_nbp = dst.mean(dim='time').mean(dim='lon') # original units
    dslten2_nbp_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    #----###-----####----- rh
    var='rh'
    ds0=load_one_model_onevar(model,runlist_wc[0],var) #load one model one experiment one var
    # ds0=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds0,var) # annual mean    
    # del ds # drop full grid from memory   
    dst= select_time_slice(dsw, 0, 9) # first ten years of flat10
    dsften_rh = dst.mean(dim='time').mean(dim='lon') # original units
    dsften_rh_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time') # converted to PgC
    dst=select_time_slice(dsw, 90, 99) # last ten years of flat10
    dslten0_rh = dst.mean(dim='time').mean(dim='lon') # original units
    dslten0_rh_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds1=load_one_model_onevar(model,runlist_wc[1],var) #load one model one experiment one var
    # ds1=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds1,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) #last ten years of flat10-zec
    dslten1_rh = dst.mean(dim='time').mean(dim='lon') # original units
    dslten1_rh_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC

    ds2=load_one_model_onevar(model,runlist_wc[2],var) #load one model one experiment one var
    # ds2=ds.mean(dim='lon') # zonal mean
    dsw=weighted_temporal_mean(ds2,var)
    # del ds # drop full grid from memory
    dst = select_time_slice(dsw, 190, 199) # last ten years of flat10-cdr
    dslten2_rh = dst.mean(dim='time').mean(dim='lon') # original units
    dslten2_rh_PgC = unitconvert*(dst*landarea).sum(dim='lon').mean(dim='time')# converted to PgC
    
    #  drop some things from memory
    del ds1, ds2, ds0

    #--------------------Save a netcdf file for each model ----------#
    # # ---- Things to save ---- #
    # variables = cTot, cVeg, cSoil, cLitter
    # timepoints = dsften, dslten0, dslten1, dslten2
    # varname = timepoints_variables_PgC
    # lat (custom to each model)

    # dsmerge=dsften_cTot_PgC.copy()

    # first make datasets .to_dataset(name='cTot')
    var='cVeg'
    dsften = dsften_cVeg_PgC.to_dataset(name='dsften_' +var +'_PgC')
    dslten0= dslten0_cVeg_PgC.to_dataset(name='dslten0_' +var +'_PgC')
    dslten1= dslten1_cVeg_PgC.to_dataset(name='dslten1_' +var +'_PgC')
    dslten2= dslten2_cVeg_PgC.to_dataset(name='dslten2_' +var +'_PgC')

    dsmerge = xr.merge([dsften, dslten0, dslten1, dslten2],compat='override')

    var = 'cSoil'
    dsften = dsften_cSoil_PgC.to_dataset(name='dsften_' +var +'_PgC')
    dslten0= dslten0_cSoil_PgC.to_dataset(name='dslten0_' +var +'_PgC')
    dslten1= dslten1_cSoil_PgC.to_dataset(name='dslten1_' +var +'_PgC')
    dslten2= dslten2_cSoil_PgC.to_dataset(name='dslten2_' +var +'_PgC')

    dsmerge = xr.merge([dsmerge, dsften, dslten0, dslten1, dslten2],compat='override')

    var='cTot'
    dsften = dsften_cTot_PgC.to_dataset(name='dsften_' +var +'_PgC')
    dslten0= dslten0_cTot_PgC.to_dataset(name='dslten0_' +var +'_PgC')
    dslten1= dslten1_cTot_PgC.to_dataset(name='dslten1_' +var +'_PgC')
    dslten2= dslten2_cTot_PgC.to_dataset(name='dslten2_' +var +'_PgC')

    dsmerge = xr.merge([dsmerge, dsften, dslten0, dslten1, dslten2],compat='override')


    # dsmerge=xr.merge([dsften_cTot_PgC, 
    #                   dsften_cVeg_PgC,
    #                   dsften_cSoil_PgC,
    #                   dslten0_cTot_PgC, 
    #                   dslten0_cVeg_PgC,
    #                   dslten0_cSoil_PgC,
    #                   dslten1_cTot_PgC, 
    #                   dslten1_cVeg_PgC,
    #                   dslten1_cSoil_PgC,
    #                   dslten2_cTot_PgC, 
    #                   dslten2_cVeg_PgC,
    #                   dslten2_cSoil_PgC
    #                  ],compat='override')
    
    if litterflag==1: # litter exists
        var='cLitter'
        dsften = dsften_cLitter_PgC.to_dataset(name='dsften_' +var +'_PgC')
        dslten0= dslten0_cLitter_PgC.to_dataset(name='dslten0_' +var +'_PgC')
        dslten1= dslten1_cLitter_PgC.to_dataset(name='dslten1_' +var +'_PgC')
        dslten2= dslten2_cLitter_PgC.to_dataset(name='dslten2_' +var +'_PgC')
        
        dsmerge = xr.merge([dsmerge, dsften, dslten0, dslten1, dslten2],compat='override')

    # else: # litter does not exist
    var = 'gpp'
    dsften = dsften_gpp_PgC.to_dataset(name='dsften_' +var +'_PgC')
    dslten0= dslten0_gpp_PgC.to_dataset(name='dslten0_' +var +'_PgC')
    dslten1= dslten1_gpp_PgC.to_dataset(name='dslten1_' +var +'_PgC')
    dslten2= dslten2_gpp_PgC.to_dataset(name='dslten2_' +var +'_PgC')

    dsmerge = xr.merge([dsmerge, dsften, dslten0, dslten1, dslten2],compat='override')

    var = 'nbp'
    dsften = dsften_nbp_PgC.to_dataset(name='dsften_' +var +'_PgC')
    dslten0= dslten0_nbp_PgC.to_dataset(name='dslten0_' +var +'_PgC')
    dslten1= dslten1_nbp_PgC.to_dataset(name='dslten1_' +var +'_PgC')
    dslten2= dslten2_nbp_PgC.to_dataset(name='dslten2_' +var +'_PgC')

    dsmerge = xr.merge([dsmerge, dsften, dslten0, dslten1, dslten2],compat='override')

    var = 'rh'
    dsften = dsften_rh_PgC.to_dataset(name='dsften_' +var +'_PgC')
    dslten0= dslten0_rh_PgC.to_dataset(name='dslten0_' +var +'_PgC')
    dslten1= dslten1_rh_PgC.to_dataset(name='dslten1_' +var +'_PgC')
    dslten2= dslten2_rh_PgC.to_dataset(name='dslten2_' +var +'_PgC')

    dsmerge = xr.merge([dsmerge, dsften, dslten0, dslten1, dslten2],compat='override')

    
    print('Saving ' +model +' to netcdf')
    # - save the matrix to a netcdf file
    dsmerge.to_netcdf('processed_output/zonal_means_timepoints_' +model +'.nc')



Loading ACCESS-ESM1-5
loading variable: cVeg
finished loading ACCESS-ESM1-5 *lat10 cVeg
loading variable: cVeg
finished loading ACCESS-ESM1-5 *lat10 cVeg
loading variable: cVeg
finished loading ACCESS-ESM1-5 *zec cVeg
loading variable: cVeg
finished loading ACCESS-ESM1-5 *cdr cVeg
loading variable: cSoil
finished loading ACCESS-ESM1-5 *lat10 cSoil
loading variable: cSoil
finished loading ACCESS-ESM1-5 *zec cSoil
loading variable: cSoil
finished loading ACCESS-ESM1-5 *cdr cSoil
loading variable: cLitter
finished loading ACCESS-ESM1-5 *lat10 cLitter
loading variable: cLitter
finished loading ACCESS-ESM1-5 *zec cLitter
loading variable: cLitter
finished loading ACCESS-ESM1-5 *cdr cLitter
loading variable: gpp
finished loading ACCESS-ESM1-5 *lat10 gpp
loading variable: gpp
finished loading ACCESS-ESM1-5 *zec gpp
loading variable: gpp
finished loading ACCESS-ESM1-5 *cdr gpp
loading variable: nbp
finished loading ACCESS-ESM1-5 *lat10 nbp
loading variable: nbp
finished loading ACCESS-ESM1-5 *

/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid

Loading CESM2
loading variable: cVeg
finished loading CESM2 *lat10 cVeg
loading variable: cVeg
finished loading CESM2 *lat10 cVeg
loading variable: cVeg
finished loading CESM2 *zec cVeg
loading variable: cVeg
finished loading CESM2 *cdr cVeg
loading variable: cSoil
finished loading CESM2 *lat10 cSoil
loading variable: cSoil
finished loading CESM2 *zec cSoil
loading variable: cSoil
finished loading CESM2 *cdr cSoil
loading variable: cLitter
finished loading CESM2 *lat10 cLitter
loading variable: cLitter
finished loading CESM2 *zec cLitter
loading variable: cLitter
finished loading CESM2 *cdr cLitter
loading variable: gpp
finished loading CESM2 *lat10 gpp
loading variable: gpp
finished loading CESM2 *zec gpp
loading variable: gpp
finished loading CESM2 *cdr gpp
loading variable: nbp
finished loading CESM2 *lat10 nbp
loading variable: nbp
finished loading CESM2 *zec nbp
loading variable: nbp
finished loading CESM2 *cdr nbp
loading variable: rh
finished loading CESM2 *lat10 rh
loading vari

/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid

Loading GFDL-ESM4
loading variable: cVeg
finished loading GFDL-ESM4 *lat10 cVeg
loading variable: cVeg
finished loading GFDL-ESM4 *lat10 cVeg
loading variable: cVeg
finished loading GFDL-ESM4 *zec cVeg
loading variable: cVeg
finished loading GFDL-ESM4 *cdr cVeg
loading variable: cSoil
finished loading GFDL-ESM4 *lat10 cSoil
loading variable: cSoil
finished loading GFDL-ESM4 *zec cSoil
loading variable: cSoil
finished loading GFDL-ESM4 *cdr cSoil
loading variable: cLitter
finished loading GFDL-ESM4 *lat10 cLitter
loading variable: cLitter
finished loading GFDL-ESM4 *zec cLitter
loading variable: cLitter
finished loading GFDL-ESM4 *cdr cLitter
loading variable: gpp
finished loading GFDL-ESM4 *lat10 gpp
loading variable: gpp
finished loading GFDL-ESM4 *zec gpp
loading variable: gpp
finished loading GFDL-ESM4 *cdr gpp
loading variable: nbp
finished loading GFDL-ESM4 *lat10 nbp
loading variable: nbp
finished loading GFDL-ESM4 *zec nbp
loading variable: nbp
finished loading GFDL-ESM4 *cdr nb

...

/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid

Loading HadCM3LC-Bris
loading variable: cVeg
finished loading HadCM3LC-Bris *lat10 cVeg
loading variable: cVeg
finished loading HadCM3LC-Bris *lat10 cVeg
loading variable: cVeg
finished loading HadCM3LC-Bris *zec cVeg
loading variable: cVeg
finished loading HadCM3LC-Bris *cdr cVeg
loading variable: cSoil
finished loading HadCM3LC-Bris *lat10 cSoil
loading variable: cSoil
finished loading HadCM3LC-Bris *zec cSoil
loading variable: cSoil
finished loading HadCM3LC-Bris *cdr cSoil
loading variable: cLitter
no data for cLitter
loading variable: gpp
finished loading HadCM3LC-Bris *lat10 gpp
loading variable: gpp
finished loading HadCM3LC-Bris *zec gpp
loading variable: gpp
finished loading HadCM3LC-Bris *cdr gpp
loading variable: nbp
finished loading HadCM3LC-Bris *lat10 nbp
loading variable: nbp
finished loading HadCM3LC-Bris *zec nbp
loading variable: nbp
finished loading HadCM3LC-Bris *cdr nbp
loading variable: rh
finished loading HadCM3LC-Bris *lat10 rh
loading variable: rh
finished load

/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid

Loading MIROC-ES2L
loading variable: cVeg
finished loading MIROC-ES2L *lat10 cVeg
loading variable: cVeg
finished loading MIROC-ES2L *lat10 cVeg
loading variable: cVeg
finished loading MIROC-ES2L *zec cVeg
loading variable: cVeg
finished loading MIROC-ES2L *cdr cVeg
loading variable: cSoil
finished loading MIROC-ES2L *lat10 cSoil
loading variable: cSoil
finished loading MIROC-ES2L *zec cSoil
loading variable: cSoil
finished loading MIROC-ES2L *cdr cSoil
loading variable: cLitter
finished loading MIROC-ES2L *lat10 cLitter
loading variable: cLitter
finished loading MIROC-ES2L *zec cLitter
loading variable: cLitter
finished loading MIROC-ES2L *cdr cLitter
loading variable: gpp
finished loading MIROC-ES2L *lat10 gpp
loading variable: gpp
finished loading MIROC-ES2L *zec gpp
loading variable: gpp
finished loading MIROC-ES2L *cdr gpp
loading variable: nbp
finished loading MIROC-ES2L *lat10 nbp
loading variable: nbp
finished loading MIROC-ES2L *zec nbp
loading variable: nbp
finished loading M

/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: invalid

In [8]:
ds_all=xr.open_dataset('processed_output/zonal_means_timepoints_' +model +'.nc')

In [9]:
ds_all

<xarray.Dataset> Size: 15kB
Dimensions:              (lat: 64)
Coordinates:
  * lat                  (lat) float64 512B 87.86 85.1 82.31 ... -85.1 -87.86
    type                 |S4 4B ...
Data variables: (12/28)
    dsften_cVeg_PgC      (lat) float64 512B ...
    dslten0_cVeg_PgC     (lat) float64 512B ...
    dslten1_cVeg_PgC     (lat) float64 512B ...
    dslten2_cVeg_PgC     (lat) float64 512B ...
    dsften_cSoil_PgC     (lat) float64 512B ...
    dslten0_cSoil_PgC    (lat) float64 512B ...
    ...                   ...
    dslten1_nbp_PgC      (lat) float64 512B ...
    dslten2_nbp_PgC      (lat) float64 512B ...
    dsften_rh_PgC        (lat) float64 512B ...
    dslten0_rh_PgC       (lat) float64 512B ...
    dslten1_rh_PgC       (lat) float64 512B ...
    dslten2_rh_PgC       (lat) float64 512B ...